In [1]:
import pandas as pd
import numpy as np
import warnings
from scipy import stats
from IPython.display import display, HTML
from sklearn import metrics as me
from sklearn.preprocessing import label_binarize
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows",20)
pd.set_option('precision', 4)
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import OneHotEncoder
from itertools import cycle
from matplotlib import cm

%pdb
%matplotlib inline

Automatic pdb calling has been turned ON


In [2]:
cmap = cm.get_cmap('Blues_r')
matplotlib.style.use('ggplot')
#print(plt.style.available)
matplotlib.rcParams['font.family'] = "Times New Roman"
matplotlib.rcParams.update({'figure.autolayout': True})


In [3]:


import itertools
#matplotlib.rcParams['figure.figsize'] = (15.0, 5.0)


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          append = "",
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    title='Confusion matrix {}'.format(append)
    np.set_printoptions(precision=4)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        #print("Normalized confusion matrix")
    else:
        #print('Confusion matrix, without normalization')
        pass
    
    #print(cm)

    label = [["\n True Negative", "\n False Positive "],
             ["\n False Negative ", "\n True Positive"]
            ]
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        
        plt.text(j, i, "{} {}".format(cm[i, j].round(4), label[i][j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

def plot(actual_value, pred_value, scenario):
    from sklearn.metrics import confusion_matrix
    append = '{}; \n Total Normal traffic:{}, \n Total Attack Traffic: {}'.format(scenario, 
                                                                            actual_value.groupby(actual_value).count()[0], 
                                                                           actual_value.groupby(actual_value).count()[1])
    
    cm_2labels = confusion_matrix(y_pred = pred_value, y_true = actual_value)
    plt.figure(figsize=[6,6])
    plot_confusion_matrix(cm_2labels, ['Normal', 'Attack'], normalize = False, append = append)


In [ ]:
result_config={}

def evaluate_lstm(model, past_scores, predictions):
    return evaluate(model, past_scores, predictions, 'LSTM')

lstm_result = []

def display_and_save(name, plot):
    fig = plot.get_figure()
    #fig.savefig("result_plots/{}.eps".format(name.replace(":","").strip()), format='eps', dpi=1000)
    #fig.savefig("result_plots/{}.png".format(name.replace(":","").strip()), format='png', dpi=1000)
    display(plot)

def evaluate(model, past_scores, predictions, model_type='AE'):
    all_scenarios = pd.DataFrame(columns=['Model','Scenarios', 'Number of Features', 'Accuracy', 'Quality Score', 'Precision', 'Recall'])

    
    def get_best_df(past_scores):
        psg = past_scores.sort_values(by='quality_score', ascending=False).groupby(by=['no_of_features', 'hidden_layers'])
        df = psg.first().sort_values(by='quality_score', ascending=False)
        #df = past_scores.sort_values(by='key')#[past_scores.key == '20151201']

        return df
    def get_median_df(past_scores):
        psg = past_scores.sort_values(by='quality_score', ascending=False).groupby(by=['no_of_features', 'hidden_layers'])
        df = psg.nth(int(psg.size()[0]/2)).sort_values(by='quality_score', ascending=False)
        return df
    def get_worst_df(past_scores):
        psg = past_scores.sort_values(by='quality_score', ascending=False).groupby(by=['no_of_features', 'hidden_layers'])
        df = psg.last().sort_values(by='quality_score', ascending=False)
        return df
    
    def get_result(past_scores, which='best'):
        if which == 'best':
            df = get_best_df(past_scores)
        elif which == 'median':
            df = get_median_df(past_scores)
        elif which == 'worst':
            df = get_worst_df(past_scores)
            
        #epoch_nof_hidden
        key = "20151221" #int(df.iloc[0]['key'])
        nof = df.iloc[0].name[0]
        hidden = df.iloc[0].name[1]
        
        return "{}_{}_{}".format(key, nof, hidden), nof, df

    def view_data(name, past_scores, which):
        _, _, df = get_result(past_scores, which)
        display(name)
        display(df)
        
        group_by = 'no_of_features'
        if(model_type == 'LSTM'):
            group_by = 'hidden_layers'
        df1 = df.reset_index().sort_values(by='quality_score', ascending=False).groupby(by=[group_by])
        
        df1 = df1.first().loc[:,['quality_score', 'time_taken']]
        df1 = df1.rename(index={1:"One", 4:"10%", 8:"20%", 16:"40%", 42:"All"})
        plot = df1.plot(secondary_y = 'time_taken', title=name)#,figsize=(10, 10))
        display_and_save(name, plot)
        return df1
        
    #display("Individual Results for each Scenario:-")    
    r_best = view_data("   Results for {}: Best case".format(model),past_scores, 'best')
    r_avg = view_data("   Results for {}: General case".format(model),past_scores,'median')
    r_worst = view_data("   Results for {}: Worst case".format(model),past_scores,'worst')
    
    global result_config
    result_config = {"Best":r_best, "Average":r_avg, "Worst":r_worst}
    
    def get_score(y_true, y_pred):
        f1 = me.matthews_corrcoef(y_true, y_pred)
        pre = me.precision_score(y_true, y_pred)
        rec = me.recall_score(y_true, y_pred)
        acc = me.accuracy_score(y_true, y_pred)
        #plot_auc(y_true, y_pred)
        return {"Quality Score":f1, "Precision":pre, "Recall":rec, "Accuracy":acc}
    
    display("Combined Results from all Scenarios for {}".format(model))

    def plot_auc(y_true, y_pred):
        y_true = y_true.astype(type(""))
        y_pred = y_pred.astype(type(""))
        
        y_true_oh = pd.get_dummies(y_true).values
        y_pred_oh = pd.get_dummies(y_pred.astype('category', categories = [0,1])).values
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        
        for i in [0,1]:
            fpr[i], tpr[i], _ = me.roc_curve(y_true_oh[:,i], y_pred_oh[:,i])
            roc_auc[i] = me.auc(fpr[i], tpr[i])
        fpr["micro"], tpr["micro"], _ = me.roc_curve(y_true_oh.ravel(), y_pred_oh.ravel())
        roc_auc["micro"] = me.auc(fpr["micro"], tpr["micro"])
        plt.figure()
        lw = 2
        colors = cycle(['aqua', 'cornflowerblue'])
        for i, color in zip([0,1], colors):
            plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                     label='ROC curve for {0} (area = {1:0.2f})'
                     ''.format("Normal" if i == 0 else "Attack", roc_auc[i]))
        plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC-AUC Curve for {}'.format(model))
        plt.legend(loc="lower right")
        display(plt.show())
        
    def accumulate_scenarios(predictions, past_scores, which='best'):
        key, nof, df = get_result(past_scores, which)
        key="20151221_1_1"
        y_true = predictions[key]["Actual"]
        y_pred = predictions[key]["Prediction"]
        y_true = y_true.dropna()
        y_pred = y_pred.dropna()
        
        scores = get_score(y_true, y_pred)
        scores.update({"Scenarios":scenario,"Number of Features":nof,"Model":model, "Duration(secs)": df.iloc[0]['time_taken']})
        plot(y_true, y_pred, "{}_{}".format(scenario, key))
        return pd.DataFrame(scores, index=[1])
    
    scenario = "Best Result"
    all_scenarios = all_scenarios.append(accumulate_scenarios(predictions, past_scores, 'best'))
    scenario = "Average Result"
    all_scenarios = all_scenarios.append(accumulate_scenarios(predictions, past_scores, 'median'))
    scenario = "Worst Result"
    all_scenarios = all_scenarios.append(accumulate_scenarios(predictions, past_scores, 'worst'))
    
    
    display(all_scenarios.set_index(['Model','Scenarios','Number of Features']))

    return all_scenarios
    
    
    
    
    

In [ ]:
past_scores = pd.read_pickle("dataset/scores/tf_dense_only_nsl_kdd_scores_all.pkl")
predictions = pd.read_pickle("dataset/tf_dense_only_nsl_kdd_predictions.pkl")


In [ ]:
all_scenarios_fcn = evaluate("FCN", past_scores, predictions)

psg = past_scores.sort_values(by='f1_score', ascending=False).groupby(by=['no_of_features', 'hidden_layers'])
df = psg.first().sort_values(by='f1_score', ascending=False).head(5)
df1 = df.reset_index().sort_values(by='f1_score', ascending=False).groupby(by=['no_of_features'])
df1 = df1.first().loc[:,['f1_score', 'time_taken']]
df1 = df1.rename(index={1:"One", 4:"10%", 8:"20%", 16:"40%", 42:"All"})
df1.plot(secondary_y = 'time_taken', table=True)

In [ ]:
past_scores = pd.read_pickle("dataset/scores/tf_vae_dense_trained_together_nsl_kdd_scores-all.pkl")
predictions = pd.read_pickle("dataset/tf_vae_dense_trained_together_nsl_kdd_predictions.pkl")

In [ ]:
all_scenarios_vae_sm = evaluate("VAE-Softmax", past_scores, predictions)

In [ ]:
past_scores = pd.read_pickle("dataset/scores/tf_vae_only_vae_loss_nsl_kdd_scores_all.pkl")
predictions = pd.read_pickle("dataset/tf_vae_only_vae_loss_nsl_kdd_predictions.pkl")

In [ ]:
all_scenarios_vae = evaluate("VAE-Label", past_scores, predictions)

In [ ]:
past_scores = pd.read_pickle("dataset/scores/tf_lstm_nsl_kdd-orig_all.pkl")
predictions = pd.read_pickle("dataset/tf_lstm_nsl_kdd_predictions.pkl")

In [ ]:
all_scenarios_lstm = evaluate_lstm("LSTM-Seq2Seq", past_scores, predictions)

In [ ]:
t = pd.concat(result_config.values(),axis=0, keys=result_config.keys())
t = t.reset_index()
t.hidden_layers = t.hidden_layers.replace({"One":"Shallow", 3:"Deep"})
t.rename(columns={"quality_score":"MCC"}, inplace=True)
t = t.pivot_table(values='MCC', index='level_0', columns='hidden_layers')
#t = t.reindex(["Shallow", "Deep"])
#t = t.loc[:,['Shallow','Deep']]

In [ ]:
ax = t.plot(kind='bar', rot=0,  legend=False, figsize=[10,4], cmap=cmap, title=False, fontsize='xx-large',
                                   edgecolor='black')

lgd = ax.legend(bbox_to_anchor=(0,1.02,1,0.2), loc="lower left",
                 ncol=3, fontsize='xx-large')
ax.set_xlabel("")
ax.set_ylabel("MCC", fontsize='xx-large')
ax.get_figure().savefig("result_plots/LSTM_Type_Kyoto.png", format='png', dpi=200, bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
t

In [ ]:
all_scenarios = pd.concat([all_scenarios_fcn, all_scenarios_vae, all_scenarios_vae_sm,  all_scenarios_lstm],axis=0)

In [ ]:
all_scenarios_display = all_scenarios.set_index(['Model', 'Scenarios', 'Number of Features'])
#all_scenarios_display

In [ ]:
all_scenarios_best = all_scenarios.set_index(['Scenarios', 'Model']) #, 'Number of Features'
all_scenarios_best.loc['Best Result']


In [ ]:
plot = all_scenarios_best.loc['Best Result'].drop('Number of Features', axis = 1).plot()
display_and_save("all_models_score_best_result", plot)

In [ ]:
all_scenarios_best.loc['Average Result']

In [ ]:
plot = all_scenarios_best.loc['Average Result'].drop('Number of Features', axis = 1).plot()
display_and_save("all_models_score_general_result", plot)

In [ ]:
all_scenarios_best.loc['Worst Result']

In [ ]:
plot = all_scenarios_best.loc['Worst Result'].drop('Number of Features', axis = 1).plot()
display_and_save("all_models_score_worst_result", plot)

In [ ]:
all_scenarios

In [ ]:
cmap = cm.get_cmap('Blues_r')
matplotlib.style.use('ggplot')
#print(plt.style.available)
matplotlib.rcParams['font.family'] = "Times New Roman"
#matplotlib.rcParams['axes.titlesize'] = 15


In [ ]:
all_scenarios_display = all_scenarios.loc[:,[ 'Scenarios', 'Quality Score', 'Model']]
all_scenarios_pivot = all_scenarios_display.pivot_table('Quality Score', 'Model', 'Scenarios')

In [ ]:
#matplotlib.style.use('fivethirtyeight')
#print(plt.style.available)
#plot_bar = all_scenarios_pivot.plot(kind='bar', figsize=[10,4], style='seaborn-colorblind', colormap=plt.cmaps.Qualitative)
#plot_bar.set_ylabel("Quality Score")
#display_and_save("All Results with Models in X Axis",plot_bar)
ax = all_scenarios_pivot.plot(kind='bar', rot=0,  legend=False, figsize=[10,4], cmap=cmap, title=False, fontsize='xx-large') 
ax.legend(bbox_to_anchor=(0,1.02,1,0.2), loc="lower left", mode="expand", ncol=3, fontsize='x-large')
ax.set_xlabel("")
display_and_save("All Results with Models in X Axis",ax)

In [ ]:
all_scenarios_display = all_scenarios.loc[:,[ 'Scenarios', 'Quality Score', 'Model']]
all_scenarios_pivot = all_scenarios_display.pivot_table('Quality Score', 'Scenarios', 'Model' )
all_scenarios_pivot

In [ ]:
plot_bar = all_scenarios_pivot.plot(kind='bar', rot=0,  legend=False, figsize=[10,4], cmap=cmap, title=False, fontsize='xx-large')
#plot_bar.set_ylabel("Quality Score")
plot_bar.legend(bbox_to_anchor=(0,1.02,1,0.2), loc="lower left", mode="expand", ncol=3, fontsize='x-large')
plot_bar.set_xlabel("")
plot_bar.set_yticks([-1,1])
plot_bar.axhline(ls='dashed', color = 'black', linewidth=1)
display_and_save("All Results with Scenarios in X Axis",plot_bar)

In [ ]:
all_scenarios_display = all_scenarios.loc[:,[ 'Scenarios', 'Duration(secs)', 'Model']]
all_scenarios_pivot = all_scenarios_display.pivot_table('Duration(secs)', 'Scenarios', 'Model' )
#all_scenarios_pivot = all_scenarios_pivot.rename(index={"LSTM-Seq2Seq":"Deep LSTM"})
all_scenarios_pivot

In [ ]:
#t.rename(index={"Average":"Average Result", "Best":"Best Result","Worst":"Worst Result"},
                columns={"Deep":"Deep LSTM", "Shallow": "Shallow LSTM"}, inplace=True)
#t1 = t.reset_index().pivot_table(values="Shallow LSTM", columns="level_0")
#t1

In [ ]:
#all_scenarios_pivot = pd.concat([all_scenarios_pivot, t1], axis = 0)
#all_scenarios_pivot

In [ ]:
ax = all_scenarios_pivot.plot(kind='bar', rot=0,  legend=False, figsize=[10,4], cmap=cmap, title=False, fontsize='xx-large') 
ax.legend(bbox_to_anchor=(0,1.02,1,0.2), loc="lower left", mode="expand", ncol=3, fontsize='x-large')
ax.set_xlabel("")
display_and_save("All Duration(secs) with Scenarios in X Axis",plot_bar)

In [ ]:
%%bash
zip -r result_plots.zip result_plots